In [213]:
from pandas import Series
import pandas as pd
import numpy as np
import collections




import matplotlib.pyplot as plt
import seaborn as sns



from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import (
    recall_score,
    precision_score,
    f1_score,
    log_loss,
    confusion_matrix,
    precision_recall_curve,
    average_precision_score,
    roc_auc_score,
    roc_curve
)
from sklearn.model_selection import (GridSearchCV,
                                     StratifiedKFold,
                                     train_test_split,
                                     KFold,
                                     StratifiedShuffleSplit,
                                     RepeatedStratifiedKFold)
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import f_classif, mutual_info_classif
from gauss_rank_scaler import GaussRankScaler

pd.set_option('display.max_rows', 50)  # показывать больше строк
pd.set_option('display.max_columns', 50)  # показывать больше колонок

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

def outliers(data): # Функция для работы с выбросами
    quantiles = np.percentile(data, [25, 75])
    iqr = quantiles[1] - quantiles[0]
    left = quantiles[0] - 1.5 * iqr
    right = quantiles[1] + 1.5 * iqr
    return left, right
    
    
    
def ohe(train, test, column): # One-hot encoding
    test['default'] = None
    full = pd.concat([train, test], axis=0)
    dummies = pd.get_dummies(full[[column]], prefix=column, drop_first=True)
    full = pd.concat([full, dummies], axis=1).drop(columns=column)
    train = full[:train.shape[0]]
    test = full[train.shape[0]:]
    test = test.drop(columns=['default'])
    return train, test

def std_scale(train, test, column): #нормализация
    scaler = StandardScaler()
    train[column] = scaler.fit_transform(train[[column]])
    test[column] = scaler.transform(test[[column]])
    
    return train, test


In [214]:
RANDOM_SEED = 42
!pip freeze > requirements.txt

In [215]:
train_1 = pd.read_csv('Project_NO/train.csv')
test_1 = pd.read_csv('Project_NO/test.csv')
sample_submission = pd.read_csv('Project_NO/sample_submission.csv')

In [216]:
train = train_1.copy()
test = test_1.copy()

Описания полей датасета

- client_id - идентификатор клиента
- education - уровень образования
- sex - пол заемщика
- age - возраст заемщика
- car - флаг наличия автомобиля
- car_type - флаг автомобиля иномарки
- decline_app_cnt - количество отказанных прошлых заявок
- good_work - флаг наличия “хорошей” работы
- bki_request_cnt - количество запросов в БКИ
- home_address - категоризатор домашнего адреса
- work_address - категоризатор рабочего адреса
- income - доход заемщика
- foreign_passport - наличие загранпаспорта
- sna - связь заемщика с клиентами банка
- first_time - давность наличия информации о заемщике
- score_bki - скоринговый балл по данным из БКИ
- region_rating - рейтинг региона
- app_date - дата подачи заявки
- default - флаг дефолта по кредиту

Посмотрим на тип данных и количество пропусков.

In [217]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73799 entries, 0 to 73798
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   client_id         73799 non-null  int64  
 1   app_date          73799 non-null  object 
 2   education         73492 non-null  object 
 3   sex               73799 non-null  object 
 4   age               73799 non-null  int64  
 5   car               73799 non-null  object 
 6   car_type          73799 non-null  object 
 7   decline_app_cnt   73799 non-null  int64  
 8   good_work         73799 non-null  int64  
 9   score_bki         73799 non-null  float64
 10  bki_request_cnt   73799 non-null  int64  
 11  region_rating     73799 non-null  int64  
 12  home_address      73799 non-null  int64  
 13  work_address      73799 non-null  int64  
 14  income            73799 non-null  int64  
 15  sna               73799 non-null  int64  
 16  first_time        73799 non-null  int64 

In [218]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36349 entries, 0 to 36348
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   client_id         36349 non-null  int64  
 1   app_date          36349 non-null  object 
 2   education         36178 non-null  object 
 3   sex               36349 non-null  object 
 4   age               36349 non-null  int64  
 5   car               36349 non-null  object 
 6   car_type          36349 non-null  object 
 7   decline_app_cnt   36349 non-null  int64  
 8   good_work         36349 non-null  int64  
 9   score_bki         36349 non-null  float64
 10  bki_request_cnt   36349 non-null  int64  
 11  region_rating     36349 non-null  int64  
 12  home_address      36349 non-null  int64  
 13  work_address      36349 non-null  int64  
 14  income            36349 non-null  int64  
 15  sna               36349 non-null  int64  
 16  first_time        36349 non-null  int64 

В данных о клиентах присутствуют переменные всех типов, не все имеют числовую запись, а также есть пропущенные значения. Значит, нам необходима предварительная очистка данных.

Пропуски встречаются только в информации об образовании клиентов. 

## FEATURE ENGINEERING

In [219]:
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']
cat_cols = [
    'education', 'home_address', 'work_address', 'sna', 'first_time', 
]
num_cols = [
    'age', 'decline_app_cnt', 'score_bki', 'bki_request_cnt', 'income', 'region_rating'
]

## Преобразование бинарных и категориальных переменных

## client_id

In [220]:
train.client_id.value_counts()

6141     1
42113    1
29467    1
19228    1
21279    1
        ..
46588    1
48637    1
42494    1
87552    1
2049     1
Name: client_id, Length: 73799, dtype: int64

Все id уникальны. Так как client_id не несёт смысловой нагрузки и не участвует в обучении модели, можем этот признак исключить.

In [221]:
# Добавим ещё признаков

#Два нижележащих повышают f1-score на 0,01, но только вместе.
train['new_one'] = (train['score_bki']/train['age'])**train['sna']
test['new_one'] = (test['score_bki']/test['age'])**test['sna']
num_cols.append('new_one')

train['new_one1'] = (train['score_bki']/train['age'])**train['foreign_passport']
test['new_one1'] = (test['score_bki']/test['age'])**test['foreign_passport']
num_cols.append('new_one1')

TypeError: unsupported operand type(s) for ** or pow(): 'float' and 'str'

In [ ]:
train.columns

## app_date

Переведём даты в удобный формат.

In [ ]:
train.app_date = pd.to_datetime(train.app_date)
test.app_date = pd.to_datetime(test.app_date)
print(train.app_date.head(5))
print(test.app_date.head(5))

In [ ]:
train.app_date.dt.month_name().value_counts()

Создадим новый признак по месяцу.

In [ ]:
train['month'] = train['app_date'].dt.month
test['month'] = test['app_date'].dt.month

train = pd.get_dummies(train, columns=['month'], drop_first=True)
test = pd.get_dummies(test, columns=['month'], drop_first=True)

Посмотрим распределение по дням недели и так же сделаем признак.

In [ ]:
train.app_date.dt.day_name().value_counts()

In [ ]:
#train['weekday'] = train['app_date'].dt.dayofweek

#test['weekday'] = test['app_date'].dt.dayofweek

#train = pd.get_dummies(train, columns=['weekday'], drop_first=True)
#test = pd.get_dummies(test, columns=['weekday'], drop_first=True)

С датами мы закончили, столбец удаляем.

In [ ]:
train.drop(columns=['app_date'], inplace=True)
test.drop(columns=['app_date'], inplace=True)

In [ ]:
train.columns

## education

In [ ]:
sns.countplot(train['education'])

Можем сделать вывод, что большая часть заёмщиков имеет образование SCH. Заполним пропуски модой.

In [ ]:
education_1 = collections.Counter(train['education'])
education_2 = collections.Counter(test['education'])

train['education'].fillna(education_1.most_common()[0][0], inplace=True)
test['education'].fillna(education_2.most_common()[0][0], inplace=True)

In [ ]:
train.columns

Применим One-hot encoding для категориальных и бинарных признаков

In [ ]:
train.head()

In [ ]:
for col in ['education', 'sex', 'car', 'car_type', 'good_work', 'foreign_passport']: 
    train, test = ohe(train, test, col)

In [ ]:
train.columns

In [ ]:
train.head()

In [ ]:
train.info()

## default

Оценим распределение целевой переменной - default. 

In [ ]:
sns.countplot(x='default', data=train)

Распределение неравномерно, поробуем это поправить с помощью oversampling.

In [ ]:
train_0 = train.query('default == 0')
train_1 = train.query('default == 1')
koeff = int(len(train_0)/len(train_1))
for i in range(koeff):
    train = train.append(train_1).reset_index(drop=True)  

In [ ]:
sns.countplot(x='default', data=train)

In [ ]:
train.columns

Посмотрим на числовые признаки

In [ ]:
for i in num_cols:
    plt.figure()
    sns.distplot(train[i][train[i] > 0].dropna(), kde = False, rug=False)
    plt.title(i)
    plt.show()

## ВЫБРОСЫ

Для оценки выбросов посмотрим на числовые признаки - age, decline_app_cnt, bki_request_cnt, income, region_rating.

## age

In [ ]:
sns.boxplot(x=train.default, y=train.age)

В среднем видим, что дефолтные клиенты младше. Выбросов нет.

## decline_app_cnt

In [ ]:
sns.boxplot(x=train.default, y=train.decline_app_cnt)

У дефолтных клиентов количество отклоненных заявок выше, чем у недефолтных. Есть выбросы, но удалять их не стоит, т.к. медиана у равна практически 0.

In [ ]:
sns.boxplot(x=train.default, y=train.bki_request_cnt)

Выбросы есть, надо убирать.

In [ ]:
sns.boxplot(x=train.default, y=train.income)

Выбросы есть, надо убирать.

In [ ]:
sns.boxplot(x=train.default, y=train.region_rating)

Удалим выбросы.

In [ ]:
num_cols_1 = ['bki_request_cnt', 'score_bki'] #Удаление выбросов только двух параметров оказало положительное влияние на модель
for colname in num_cols_1:
    d = outliers(train[colname])
    train = train.loc[train[colname].between(d[0], d[1])]

In [ ]:
train.head()

## Посмотрим на графики распределения числовых переменных:

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(25, 12))
for col, i in zip(num_cols, range(8)):
    sns.distplot(train[col], kde=False, ax=axes.flat[i])

In [ ]:
num_cols_log = ['age', 'decline_app_cnt', 'bki_request_cnt', 'income', 'region_rating']

for i in num_cols_log:
    train[i] = np.log(train[i] + 1)

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(25, 12))
for col, i in zip(num_cols, range(8)):
    sns.distplot(train[col], kde=False, ax=axes.flat[i])

In [ ]:
for i in num_cols_log:
    test[i] = np.log(test[i] + 1)

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(25, 12))
for col, i in zip(num_cols, range(7)):
    sns.distplot(test[col], kde=False, ax=axes.flat[i])

## Нормализация

In [ ]:
for col in ['income', 'bki_request_cnt', 'decline_app_cnt', 'age', 
            'region_rating', 'sna', 'home_address', 'work_address', 'first_time']:
     train, test = std_scale(train, test, col)

In [ ]:
train.head()

In [ ]:
train.drop(columns=['client_id'], axis=0, inplace=True)
test.drop(columns=['client_id'], axis=0, inplace=True)


In [ ]:
train.shape


In [ ]:
test.shape


## Model_training

In [ ]:
X = train.drop(columns='default').values
y = train.default.values.astype(int)

In [ ]:
param_grid = {
    'penalty': ['l2'],
    'C': np.logspace(-2, 2, 10),
}

In [ ]:
s = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
train_ind, val_ind = list(s.split(X, y))[0]

In [ ]:
X_train, X_val, y_train, y_val = X[train_ind], X[val_ind], y[train_ind], y[val_ind]

In [ ]:
param_grid.update({'class_weight': ['balanced', {0: 1, 1: 10}]})

In [ ]:
param_grid

In [ ]:
stratified_cv = RepeatedStratifiedKFold(n_repeats=2, random_state=RANDOM_SEED)

In [ ]:
logreg = LogisticRegression(max_iter=500, class_weight='balanced')
gs = GridSearchCV(logreg, param_grid=param_grid, cv=stratified_cv, scoring='roc_auc')

In [ ]:
gs.fit(X_train, y_train)

In [ ]:
gs.best_params_

In [ ]:
probs = gs.predict_proba(X_val)
preds = gs.predict(X_val)
print('ROC AUC', roc_auc_score(y_val, probs[:, 1]))
print()
print('Confusion matrix: \n', confusion_matrix(y_val, preds))

In [ ]:
probs = gs.predict_proba(test)
probs = probs[:, 1]

In [ ]:
test['client_id'] = test_1['client_id']

In [ ]:
client_id = test['client_id']

In [ ]:
my_submission = pd.DataFrame({'client_id': client_id,
                              'default': probs})
my_submission.to_csv('submission.csv', index=False)

my_submission.sample(10)

## ВЫВОДЫ

По итогам работы с данными можно сделать вывод, что наиболее сильное влияние на силу предсказания оказывает EDA и Feature Engeneering. Тем не менее, логарифмирование и нормализция также оказали положительное влияние на модель.